## Generate routes

**Author**: Imad Saddik
<br/>
**Date**: 21/12/2024

---

**Table of contents**<a id='toc0_'></a>    
- [Load nodes](#toc1_1_)    
- [Route between 2 points](#toc1_2_)    
  - [Project OSRM](#toc1_2_1_)    
- [Save routes between all nodes](#toc1_3_)    

---

## <a id='toc1_1_'></a>[Load nodes](#toc0_)

In [1]:
import psycopg2

connection = psycopg2.connect(
    dbname='routes',
    user='postgres',
    password='postgres',
    host='localhost'
)
connection.autocommit = True
cursor = connection.cursor()

In [2]:
from pydantic import BaseModel


class Node(BaseModel):
    id: int
    latitude: float
    longitude: float
    type_: str

In [3]:
response = cursor.execute("""
SELECT * FROM nodes
""")

data = cursor.fetchall()
nodes = [Node(id=row[0], latitude=row[1], longitude=row[2], type_=row[3])
         for row in data]

In [4]:
bus_nodes = [node for node in nodes if node.type_ == 'bus']
employee_nodes = [node for node in nodes if node.type_ == 'employee']
company_nodes = [node for node in nodes if node.type_ == 'company']

len(bus_nodes), len(employee_nodes), len(company_nodes)

(40, 500, 1)

In [ ]:
bus_ids = [node.id for node in bus_nodes]
employee_ids = [node.id for node in employee_nodes]
company_ids = [node.id for node in company_nodes]

## <a id='toc1_2_'></a>[Route between 2 points](#toc0_)

### <a id='toc1_2_1_'></a>[Project OSRM](#toc0_)

Cons : Rate limited (1RPM)
Solution : Deploy it locally

In [6]:
import folium

employee_1 = employee_nodes[0]
employee_2 = employee_nodes[1]

my_map = folium.Map(location=[employee_1.latitude,
                    employee_1.longitude], zoom_start=13)
folium.Marker([employee_1.latitude, employee_1.longitude],
              popup='Employee 1').add_to(my_map)
folium.Marker([employee_2.latitude, employee_2.longitude],
              popup='Employee 2').add_to(my_map)

my_map

In [7]:
import requests

service = 'route'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

employee_1_latitude = employee_1.latitude
employee_1_longitude = employee_1.longitude
employee_2_latitude = employee_2.latitude
employee_2_longitude = employee_2.longitude

coordinates = f'{employee_1_longitude},{employee_1_latitude};{
    employee_2_longitude},{employee_2_latitude}'
url = f"{host}/{service}/{version}/{profile}/{
    coordinates}?overview=full&steps=true&geometries=geojson"

response = requests.get(url)
response.status_code

200

In [8]:
data = response.json()
data.keys()

dict_keys(['code', 'routes', 'waypoints'])

There is one route that take us from employee 1 to employee 2

In [9]:
routes = data['routes']
len(routes)

1

In [10]:
route = routes[0]
route.keys()

dict_keys(['geometry', 'legs', 'weight_name', 'weight', 'duration', 'distance'])

We can extract the coordinates of the route, its duration in seconds and distance in meters.

In [11]:
route_coordinates = route['geometry']["coordinates"]
route_distance = route["distance"]  # unit: meters
route_duration = route["duration"]  # unit: seconds

Each point in the routes coordinates list has this format [longitude, latitude], We will convert it to [latitude, longitude]

In [12]:
route_coordinates = [[point[1], point[0]] for point in route_coordinates]

In [ ]:
folium.PolyLine(
    route_coordinates,
    color="blue",
    weight=2.5,
    opacity=1
).add_to(my_map)
my_map

## <a id='toc1_3_'></a>[Save routes between all nodes](#toc0_)

In [14]:
import psycopg2

connection = psycopg2.connect(
    dbname='routes',
    user='postgres',
    password='postgres',
    host='localhost'
)
connection.autocommit = True
cursor = connection.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS route_mapping (
    source_node_id INTEGER NOT NULL,
    destination_node_id INTEGER NOT NULL,
    duration FLOAT NOT NULL,
    distance FLOAT NOT NULL,
    coordinates JSON NOT NULL,
    PRIMARY KEY (source_node_id, destination_node_id),
    CONSTRAINT fk_source_node FOREIGN KEY (source_node_id) REFERENCES nodes (id),
    CONSTRAINT fk_destination_node FOREIGN KEY (destination_node_id) REFERENCES nodes (id)
);
""")
connection.commit()

In [15]:
nodes[:5]

[Node(id=1, latitude=35.7873286, longitude=-5.8165392, type_='bus'),
 Node(id=2, latitude=35.7661336, longitude=-5.7665911, type_='bus'),
 Node(id=3, latitude=35.7273919, longitude=-5.8584305, type_='bus'),
 Node(id=4, latitude=35.7764352, longitude=-5.8044441, type_='bus'),
 Node(id=5, latitude=35.7220829, longitude=-5.7726176, type_='bus')]

In [16]:
len(nodes) * (len(nodes) - 1) // 2

146070

In [17]:
import json

from tqdm import tqdm
from itertools import product

service = 'route'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

for source_node, destination_node in tqdm(
    product(nodes, nodes),
    desc="Processing routes",
    total=len(nodes) * len(nodes)
):
    source_node_id = source_node.id
    destination_node_id = destination_node.id

    if source_node_id == destination_node_id:
        continue

    coordinates = f"{source_node.longitude},{source_node.latitude};{
        destination_node.longitude},{destination_node.latitude}"
    url = f"{host}/{service}/{version}/{profile}/{
        coordinates}?overview=full&steps=true&geometries=geojson"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        route = data['routes'][0]

        # Extract route details
        distance = float(route['distance'])
        duration = float(route['duration'])
        coordinates_json = route['geometry']['coordinates']
        coordinates_json = [[point[1], point[0]] for point in coordinates_json]

        query = """
            INSERT INTO route_mapping (source_node_id, destination_node_id, duration, distance, coordinates)
            VALUES (%s, %s, %s, %s, %s)
            ON CONFLICT (source_node_id, destination_node_id) DO NOTHING;
        """
        params = (source_node_id, destination_node_id, duration,
                  distance, json.dumps(coordinates_json))
        cursor.execute(query, params)
    else:
        print(f"""Failed to fetch route for {
              source_node_id} -> {destination_node_id}, status code: {response.status_code}""")

connection.commit()
cursor.close()
connection.close()

print("All routes processed and saved to the database.")

Processing routes: 100%|██████████| 292681/292681 [33:53<00:00, 143.94it/s]

All routes processed and saved to the database.
